In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Visualization of the second Riemann sheet with rotated branch cut 
TR-979
^^^
To reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=4) of the PDG way to get from the first physical sheet to the second unphysical sheet is derived for the Amplitude calculated within the K matrix formalism.   
:::
::::

# Riemann sheets for two channels


In [ ]:
%pip install -q ampform==0.14.8 plotly==5.17.0 sympy==1.12

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math
from matplotlib.backends.backend_pdf import PdfPages

warnings.filterwarnings("ignore")

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr = PhaseSpaceFactor(s, m1, m2)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

In [ ]:
from ampform.kinematics.phasespace import Kallen


@unevaluated(real=False)
class PhaseSpaceFactor_Kaellen(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return 2 * BreakupMomentum(s, m1, m2) / sp.sqrt(s)


@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -(16 * sp.pi) * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr_Kaellen = PhaseSpaceFactor_Kaellen(s, m1, m2)
rho_cm_expr = PhaseSpaceCM(s, m1, m2)
cm_expr = ChewMandelstam(s, m1, m2)
q_expr = BreakupMomentum(s, m1, m2)
kallen = Kallen(*sp.symbols("x:z"))
Math(
    aslatex({
        e: e.doit(deep=False)
        for e in [rho_expr_Kaellen, rho_cm_expr, cm_expr, q_expr, kallen]
    })
)

# T matrix definition with K matrix - 2 channel case 

## First Riemann sheet 

In [ ]:
class DiagonalMatrix(sp.DiagonalMatrix):
    def _latex(self, printer, *args):
        return R"\rho"

In [ ]:
class DiagonalMatrix(sp.DiagonalMatrix):
    def _latex(self, printer, *args):
        return "\rho"


n_channels = 2
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)

CM_symbol = sp.MatrixSymbol(R"{\rho_{\Sigma}}", n_channels, n_channels)
CM = sp.DiagonalMatrix(CM_symbol)
CM.as_explicit()

In [ ]:
T_I = (I - K * CM).inv() * K
T_I

In [ ]:
T_I_explicit = T_I.as_explicit()
T_I_explicit[0, 0]

In [ ]:
CM_I_explicit = CM.as_explicit()

In [ ]:
s = sp.Symbol("s")
ma1 = sp.Symbol("m_{a1}")
mb1 = sp.Symbol("m_{b1}")
ma2 = sp.Symbol("m_{a2}")
mb2 = sp.Symbol("m_{b2}")
m0 = sp.Symbol("m0")
w0 = sp.Symbol("Gamma0")
gamma1 = sp.Symbol("gamma1")
gamma2 = sp.Symbol("gamma2")
phi_rot = sp.Symbol("phi")
wp = sp.Symbol("wp")
symbols = sp.Tuple(s, ma1, mb1, ma2, mb2, m0, w0, gamma1, gamma2)
symbols_bw = sp.Tuple(s, m0, wp)
symbols_rot = sp.Tuple(s, ma1, mb1, ma2, mb2, m0, w0, gamma1, gamma2, phi_rot)
k_expr_00 = (gamma1 * gamma1 * m0 * w0) / (m0**2 - s)
k_expr_10 = (gamma1 * gamma2 * m0 * w0) / (m0**2 - s)
k_expr_11 = (gamma2 * gamma2 * m0 * w0) / (m0**2 - s)
Bw_expr = (gamma2 * gamma2 * m0 * w0) / (m0**2 - s)

In [ ]:
cm_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    CM[0, 0]: PhaseSpaceCM(s, ma1, mb1),
    CM[1, 1]: PhaseSpaceCM(s, ma2, mb2),
}

In [ ]:
T_I_cm_expr_00 = T_I_explicit[0, 0].xreplace(cm_expressions)
T_I_cm_expr_11 = T_I_explicit[1, 1].xreplace(cm_expressions)
CM_I_expr_00 = CM_I_explicit[0, 0].xreplace(cm_expressions)

In [ ]:
T_I_cm_expr_00.simplify(doit=False)

In [ ]:
T_I_cm_expr_11.simplify(doit=False)

In [ ]:
CM_I_expr_00.simplify(doit=False)

In [ ]:
bw_expr = np.pi * wp * m0 / (m0**2 - s - wp * m0 * sp.I)

In [ ]:
T_I_func = sp.lambdify(symbols, T_I_cm_expr_00.doit())
CM_I_func = sp.lambdify(symbols, CM_I_expr_00.doit())
bw_func = sp.lambdify(symbols_bw, bw_expr.doit())
values = {
    ma1: 0.135,
    mb1: 0.55,
    ma2: 0.493,
    mb2: 0.493,
    m0: 0.98,
    w0: 0.03,
    gamma1: 1,
    gamma2: 0.5,
}
values_bw = {
    m0: 0.98,
    wp: 0.02,
}
args = eval(str(symbols[1:].xreplace(values)))
args_bw = eval(str(symbols_bw[1:].xreplace(values_bw)))
epsilon = 1e-10
x = np.linspace(0.5, 1.5, num=700)
y = np.linspace(epsilon, 5, num=700)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

T1n = T_I_func(Zn, *args)
T1p = T_I_func(Zp, *args)
CMp = CM_I_func(Zp, *args)
Bn = bw_func(x, *args_bw)

fig, axes = plt.subplots(figsize=(10, 6), ncols=1, sharey=True)
ax1 = axes
ax1.set_xlabel(R"$\mathrm{Re}(s)$")
ax1.set_ylabel(R"$\mathrm{T}(s)$")

ax1.plot(x, (T1n[0]).imag, label=R"$Im(T)$", color="blue")
ax1.plot(x, (T1n[0]).real, label=R"$Re(T)$", color="red")
ax1.plot(x, T1n[0].real + T1n[0].imag, label=R"$Abs(T)$", color="yellow")
# ax1.plot(x, np.abs(Bn), label=R"$Abs(BW)$",color='magenta',linestyle='--')
ax1.axvline(x=0.98, linestyle="--", color="grey", label=R"$m_{R1}$")
ax1.legend()
with PdfPages("K_1ch.pdf") as pdf:
    pdf.savefig(fig, dpi=1200)
    plt.close(fig)
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(5, 5), ncols=1, sharey=True)
ax1 = axes
ax1.set_xlabel(R"$\mathrm{Re}(s)$")
ax1.set_ylabel(R"$\mathrm{Im}(s)$")
ax1.set_aspect(1)
ax1.plot(T1n[0].real, (T1n[0]).imag, label=R"$T_\mathrm{I}(s-0i)$", color="blue")
# ax1.plot(Bn.real,Bn.imag, label=R"$T_\mathrm{I}(s-0i)$",color='red')
# Save the figure with higher resolution (e.g., dpi=300)
with PdfPages("K_agrand.pdf") as pdf:
    pdf.savefig(fig, dpi=1200)
fig.tight_layout()
plt.show()

# Two Poles

In [ ]:
m1 = sp.Symbol("m1")
w1 = sp.Symbol("Gamma1")
gamma11 = sp.Symbol("gamma21")
gamma22 = sp.Symbol("gamma22")
symbols_2 = sp.Tuple(
    s, ma1, mb1, ma2, mb2, m0, w0, gamma1, gamma2, m1, w1, gamma11, gamma22
)
k_expr_00_1 = (gamma1 * gamma1 * m0 * w0) / (m0**2 - s) - (
    gamma11 * gamma11 * m1 * w1
) / (m1**2 - s)
k_expr_11_1 = (gamma2 * gamma2 * m0 * w0) / (m0**2 - s) - (
    gamma22 * gamma22 * m1 * w1
) / (m1**2 - s)
k_expr_01_1 = (gamma1 * gamma2 * m0 * w0) / (m0**2 - s) - (
    gamma11 * gamma22 * m1 * w1
) / (m1**2 - s)

In [ ]:
cm_expressions_2 = {
    K[0, 0]: k_expr_00_1,
    K[1, 1]: k_expr_11_1,
    K[0, 1]: k_expr_01_1,
    K[1, 0]: k_expr_01_1,
    CM[0, 0]: PhaseSpaceCM(s, ma1, mb1),
    CM[1, 1]: PhaseSpaceCM(s, ma2, mb2),
}

In [ ]:
T_00_expr_2 = T_I_explicit[0, 0].xreplace(cm_expressions_2)
T_00_expr_2.simplify(doit=False)

In [ ]:
T_func_2 = sp.lambdify(symbols_2, T_00_expr_2.doit())
values_2 = {
    ma1: 0.135,
    mb1: 0.55,
    ma2: 0.493,
    mb2: 0.493,
    m0: 0.98,
    w0: 0.03,
    gamma1: 1,
    gamma2: 0.5,
    m1: 0.95,
    w1: 0.05,
    gamma11: 1,
    gamma22: 1,
}

args_2 = eval(str(symbols_2[1:].xreplace(values_2)))
epsilon = 1e-10
x = np.linspace(0.6, 1.2, num=700)
y = np.linspace(epsilon, 5, num=700)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

T1n_2 = T_func_2(Zn, *args_2)
T1p_2 = T_func_2(Zp, *args_2)


fig, axes = plt.subplots(figsize=(10, 6), ncols=1, sharey=True)
ax1 = axes
ax1.set_xlabel(R"$\mathrm{Re}(s)$")
ax1.set_ylabel(R"$\mathrm{T}(s)$")

ax1.plot(x, (T1n_2[0]).imag, label=R"$Im(T)$", color="blue")
ax1.plot(x, (T1n_2[0]).real, label=R"$Re(T)$", color="red")
ax1.plot(x, T1n_2[0].real + T1n_2[0].imag, label=R"$Abs(T)$", color="yellow")
ax1.axvline(x=0.98, linestyle="--", color="grey", label=R"$m_{R1}$")
ax1.axvline(x=0.89, linestyle="--", color="grey", label=R"$m_{R2}$")
ax1.legend()
with PdfPages("K_2ch.pdf") as pdf:
    pdf.savefig(fig, dpi=1200)
    plt.close(fig)
fig.tight_layout()
plt.show()
fig.tight_layout()
plt.show()